In [1]:
import numpy as np
import pandas as pd

# Load aligned amino acid sequences
with open("DataFile1.txt", "r") as f:
    sequences = [line.strip() for line in f if line.strip()]
print(sequences)

['...']


In [9]:
len(sequences[0])

120

# New Section

In [ ]:
from collections import defaultdict
# Ensure sequences are aligned and same length
sequence_length = len(sequences[0])
num_sequences = len(sequences)

# Count amino acid pairs at each position
pair_counts = defaultdict(int)
aa_counts = defaultdict(int)

for i in range(num_sequences):
    for j in range(i + 1, num_sequences):
        for pos in range(sequence_length):
            a1, a2 = sequences[i][pos], sequences[j][pos]
            pair = tuple(sorted((a1, a2)))
            pair_counts[pair] += 1
            aa_counts[a1] += 1
            aa_counts[a2] += 1

# Observed pair frequencies
total_pairs = sum(pair_counts.values())
obs_freq = {pair: count / total_pairs for pair, count in pair_counts.items()}

# Amino acid frequencies
aa_total = sum(aa_counts.values())
aa_freq = {aa: count / aa_total for aa, count in aa_counts.items()}

# Create substitution matrix using log-odds
amino_acids = sorted(set("".join(sequences)))
matrix = pd.DataFrame(index=amino_acids, columns=amino_acids, dtype=float)

for aa1 in amino_acids:
    for aa2 in amino_acids:
        pair = tuple(sorted((aa1, aa2)))
        p_ab = obs_freq.get(pair, 1e-9)
        p_a = aa_freq.get(aa1, 1e-9)
        p_b = aa_freq.get(aa2, 1e-9)
        expected = p_a * p_b * (2 if aa1 != aa2 else 1)
        score = np.log2(p_ab / expected)
        matrix.at[aa1, aa2] = round(score, 2)

matrix.fillna(0, inplace=True)
print("Substitution matrix:")
matrix



Substitution matrix:


,a,c,d,e,f,g,h,i,k,l,m,n,p,q,r,s,t,v,w,y
a,3.68,-0.04,0.92,-1.19,-1.96,-2.21,-0.34,-1.87,-1.50,-1.88,-1.67,-0.43,-1.40,-1.26,-2.08,-2.42,0.70,-0.35,-0.47,-1.97
c,-0.04,1.94,0.35,-0.70,-0.25,-0.20,-0.36,-0.76,0.33,0.37,-0.20,-0.65,-0.11,-0.80,-0.62,-0.06,-0.83,-0.27,-0.80,0.37
d,0.92,0.35,1.62,0.27,0.20,-0.17,-0.10,-0.06,0.13,-0.52,-0.44,-0.64,-0.39,-0.48,-0.51,0.16,-0.13,-0.26,-0.26,-0.28
e,-1.19,-0.70,0.27,2.08,-0.21,0.35,0.36,0.51,-0.49,-0.90,0.23,-0.33,-0.88,-0.20,-0.03,0.01,-0.66,-0.89,0.02,-0.40
f,-1.96,-0.25,0.20,-0.21,1.66,0.12,-0.22,0.26,-0.91,-0.24,0.22,-0.42,-0.09,0.11,0.37,-0.24,-0.38,-0.35,-0.43,-0.37
g,-2.21,-0.20,-0.17,0.35,0.12,2.19,-0.62,0.05,-0.12,0.15,-0.55,-0.78,-0.03,-0.76,-0.22,0.33,-0.54,-0.57,0.21,-0.84
h,-0.34,-0.36,-0.10,0.36,-0.22,-0.62,2.00,0.26,-0.21,-0.51,-0.45,0.01,-0.68,0.17,-0.05,0.22,0.49,-0.81,-0.75,0.00
i,-1.87,-0.76,-0.06,0.51,0.26,0.05,0.26,1.71,-0.17,-0.53,0.44,0.25,0.11,0.23,-0.35,0.09,-0.31,-1.02,-0.46,-0.44
k,-1.50,0.33,0.13,-0.49,-0.91,-0.12,-0.21,-0.17,1.83,-0.50,-0.39,-0.43,0.07,-0.30,0.05,-0.61,0.54,-0.48,-0.20,0.33
l,-1.88,0.37,-0.52,-0.90,-0.24,0.15,-0.51,-0.53,-0.50,2.38,-0.35,-0.53,0.11,0.30,-0.47,-0.01,-0.78,-0.41,-0.37,-0.13


In [11]:
print(dict(sorted(aa_counts.items())))
print(dict(sorted(obs_freq.items())))

{'a': 79700, 'c': 124016, 'd': 71305, 'e': 110051, 'f': 137952, 'g': 97911, 'h': 80078, 'i': 91966, 'k': 96610, 'l': 97644, 'm': 112744, 'n': 121147, 'p': 128325, 'q': 116859, 'r': 92977, 's': 121149, 't': 160240, 'v': 213228, 'w': 183803, 'y': 162295}
{('a', 'a'): 0.0141275, ('a', 'c'): 0.0033408333333333332, ('a', 'd'): 0.0037325, ('a', 'e'): 0.0013316666666666668, ('a', 'f'): 0.0009825, ('a', 'g'): 0.0005858333333333333, ('a', 'h'): 0.0017491666666666667, ('a', 'i'): 0.0006975, ('a', 'k'): 0.0009441666666666666, ('a', 'l'): 0.0007358333333333333, ('a', 'm'): 0.0009833333333333332, ('a', 'n'): 0.00249, ('a', 'p'): 0.0013458333333333334, ('a', 'q'): 0.00135, ('a', 'r'): 0.0006066666666666667, ('a', 's'): 0.0006266666666666666, ('a', 't'): 0.007193333333333334, ('a', 'v'): 0.0046391666666666664, ('a', 'w'): 0.0036783333333333334, ('a', 'y'): 0.0011483333333333332, ('c', 'c'): 0.010279166666666667, ('c', 'd'): 0.0039183333333333336, ('c', 'e'): 0.00292, ('c', 'f'): 0.005, ('c', 'g'): 0.

In [15]:
print(sequence_length)
print(num_sequences)
print(pair_counts)
print(aa_counts)
print(total_pairs)
print(obs_freq)

120
200
defaultdict(<class 'int'>, {('s', 's'): 19389, ('m', 'p'): 8007, ('p', 'p'): 27699, ('f', 's'): 11809, ('e', 'y'): 11403, ('h', 'k'): 5844, ('f', 'y'): 13388, ('f', 'f'): 25481, ('t', 't'): 33700, ('w', 'w'): 46744, ('v', 'v'): 71429, ('n', 'v'): 18522, ('g', 'g'): 17424, ('s', 'v'): 19598, ('a', 'a'): 33534, ('l', 'l'): 21752, ('a', 'k'): 2318, ('h', 'h'): 11296, ('g', 'r'): 6107, ('t', 'y'): 20140, ('f', 'k'): 5882, ('h', 'p'): 4942, ('c', 'h'): 6229, ('c', 't'): 9473, ('l', 'q'): 12326, ('e', 'i'): 12261, ('c', 'f'): 12378, ('k', 'w'): 13175, ('c', 'n'): 8857, ('p', 'r'): 11246, ('i', 'v'): 7455, ('k', 'k'): 13634, ('c', 'c'): 24926, ('g', 'y'): 6575, ('r', 'w'): 11439, ('l', 'n'): 6676, ('m', 'n'): 9765, ('g', 'p'): 11105, ('m', 'v'): 12903, ('v', 'w'): 29821, ('w', 'y'): 13983, ('y', 'y'): 41459, ('t', 'w'): 18759, ('p', 's'): 15616, ('r', 's'): 8520, ('r', 't'): 8906, ('h', 'w'): 7667, ('k', 'v'): 12901, ('i', 'i'): 12086, ('f', 'n'): 10058, ('d', 'g'): 5243, ('d', 'h'): 

In [16]:
from collections import Counter, defaultdict

# Load amino acid and label sequences
with open("DataFile2.txt", "r") as f:
    lines = [line.strip() for line in f if line.strip()]

aa_sequences = lines[::2]
state_sequences = lines[1::2]
assert len(aa_sequences) == len(state_sequences)

# Emission and transition counts
emission_counts = {state: Counter() for state in ['0', '1', '2']}
transition_counts = defaultdict(Counter)
state_totals = Counter()

for aas, states in zip(aa_sequences, state_sequences):
    for i in range(len(aas)):
        s = states[i]
        a = aas[i]
        emission_counts[s][a] += 1
        state_totals[s] += 1
        if i > 0:
            prev_s = states[i - 1]
            transition_counts[prev_s][s] += 1

# Emission probabilities
emission_probs = {}
for state, counter in emission_counts.items():
    total = state_totals[state]
    emission_probs[state] = {aa: count / total for aa, count in counter.items()}

# Transition probabilities
transition_probs = {}
for s_from, counter in transition_counts.items():
    total = sum(counter.values())
    transition_probs[s_from] = {s_to: count / total for s_to, count in counter.items()}

# Output results
import pprint
print("\nEmission Probabilities:")
pprint.pprint(emission_probs)

print("\nTransition Probabilities:")
pprint.pprint(transition_probs)



Emission Probabilities:
{'0': {'a': 0.03909590714722053,
       'c': 0.03836285888821014,
       'd': 0.018692730604764812,
       'e': 0.03909590714722053,
       'f': 0.06597434331093463,
       'g': 0.03726328649969456,
       'h': 0.03653023824068418,
       'i': 0.029077580940745265,
       'k': 0.032864996945632254,
       'l': 0.03481979230299328,
       'm': 0.046670739156994503,
       'n': 0.055833842394624314,
       'p': 0.05839951130116066,
       'q': 0.05436774587660354,
       'r': 0.028222357971899818,
       's': 0.05766646304215028,
       't': 0.0596212583995113,
       'v': 0.11350030543677458,
       'w': 0.074282223579719,
       'y': 0.07965791081246182},
 '1': {'a': 0.0793564221434415,
       'c': 0.02754295064085083,
       'd': 0.017180256340332697,
       'e': 0.02617943823288792,
       'f': 0.04963185164985001,
       'g': 0.018543768748295608,
       'h': 0.01772566130351786,
       'i': 0.021270793564221433,
       'k': 0.023997818380147258,
       'l':